In [1]:
import os

In [2]:
pwd

'c:\\Users\\DELL\\Downloads\\github\\Kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [ ]:
pwd

In [4]:
#entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:

from med_classifier.constants import *
from med_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
#config manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
#component
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        

In [10]:
#pipeline

# pipeline

try:
    import shutil
    import random

    def create_sample_dataset(source_dir, dest_dir, samples_per_class=100):
        # Clean old sample directory if it exists
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
        os.makedirs(dest_dir, exist_ok=True)

        for class_name in os.listdir(source_dir):
            class_path = os.path.join(source_dir, class_name)
            if not os.path.isdir(class_path):
                continue

            images = os.listdir(class_path)
            random.shuffle(images)
            sample_images = images[:samples_per_class]

            dest_class_path = os.path.join(dest_dir, class_name)
            os.makedirs(dest_class_path, exist_ok=True)

            for img in sample_images:
                src = os.path.join(class_path, img)
                if not os.path.isfile(src):
                    continue
                dst = os.path.join(dest_class_path, img)
                shutil.copy(src, dst)

    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()

    # x starts here
    original_data_path = training_config.training_data
    sample_data_path = os.path.join("artifacts", "data_ingestion","kidney-ct-scan-image")
    create_sample_dataset(original_data_path, sample_data_path, samples_per_class=100)

# Create new config with updated path
    from med_classifier.entity.config_entity import TrainingConfig

    new_training_config = TrainingConfig(
    root_dir=training_config.root_dir,
    trained_model_path=training_config.trained_model_path,
    updated_base_model_path=training_config.updated_base_model_path,
    training_data=Path(sample_data_path),
    params_epochs=training_config.params_epochs,
    params_batch_size=training_config.params_batch_size,
    params_is_augmentation=training_config.params_is_augmentation,
    params_image_size=training_config.params_image_size
    )
   
    training = Training(config=new_training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

  

except Exception as e:
    raise e


[2025-07-30 01:10:56,679: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-30 01:10:56,688: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-30 01:10:56,691: INFO: common: created directory at: artifacts]
[2025-07-30 01:10:56,694: INFO: common: created directory at: artifacts\training]
Found 40 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
10/10 [==============================] - 51s 5s/step - loss: 11.2807 - accuracy: 0.5375 - val_loss: 57.7204 - val_accuracy: 0.3750
